<a href="https://colab.research.google.com/github/DongHyukShin93/BigData/blob/main/210610_01_Text_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BeautifulSoap 사용하여 HTML 파싱

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

# 1) 문자열에서 soup을 생성한다.
soup1 = BeautifulSoup("<HTML><HEAD><<headers>></HEAD><<body>></HTML>")

# 2) 로컬 파일에서 soup을 생성한다. -> myDoc.html 파일을 이용
soup2 = BeautifulSoup(open("myDoc.html"))

# 3) 웹문서에서 soup을 생성한다. -> 크롤링의 기초
soup3 = BeautifulSoup(urlopen("http://www.networksciencelab.com/"))

In [ ]:
soup3

find(), find_all() 특정 속성(href=하이퍼링크 정보) 값 찾기

In [ ]:
# http://www.networksciencelab.com/ 사이트에서 하이퍼링크 정보 모두 가져오기

with urlopen("http://www.networksciencelab.com/") as doc :
  soup = BeautifulSoup(doc)

# 하이퍼링크만 추출
links = [(link.string,link["href"]) for link in soup.find_all('a') if link.has_attr("href")]
links

CSV 파일 다루기

statistics 모듈을 사용해서 나이 변수의 평균과 표준편차 구하기

데이터 가져오기

In [ ]:
import csv

with open("/content/Demographic_Statistics_By_Zip_Code.csv") as infile :
  data = list(csv.reader(infile)) # data = list(csv.reader(open("/content/Demographic_Statistics_By_Zip_Code.csv")))

In [ ]:
# 첫번째 레코드에 COUNT PARTICIPANTS 데이터 인덱스 값 추출 data[0] 컬럼 값
countParticipantsIndex = data[0].index("COUNT PARTICIPANTS")
countParticipantsIndex

1

In [ ]:
# COUNT PARTICIPANTS 데이터 추출
countParticipants = [int(row[countParticipantsIndex]) for row in data[1:]] # 리스트 내포

나이의 평균과 표준편차

In [ ]:
import statistics

print(statistics.mean(countParticipants), statistics.stdev(countParticipants)) # stdev() : 표준편차 구하는 함수

17.661016949152543 43.27973735299687


자연어 처리하기

In [ ]:
pip install nltk

In [ ]:
import nltk
nltk.__version__

'3.2.5'

nltk 에서 영단어 온톨로지(wordnet) 사용하기

In [ ]:
# https://frhyme.github.io/python-lib/nltk-wordnet/
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
wn = nltk.corpus.wordnet # 코퍼스 리더(reader)

# synset : 유의어 묶음으로 단어-품사-순번 형태로 구성
wn.synsets('cat') # synset's'

[Synset('cat.n.01'),
 Synset('guy.n.01'),
 Synset('cat.n.03'),
 Synset('kat.n.01'),
 Synset('cat-o'-nine-tails.n.01'),
 Synset('caterpillar.n.02'),
 Synset('big_cat.n.01'),
 Synset('computerized_tomography.n.01'),
 Synset('cat.v.01'),
 Synset('vomit.v.01')]

In [ ]:
wn.synset('cat.n.01').hypernyms() # hypernyms 상위어
wn.synset('cat.n.01').hyponyms() # hyponyms 하위어

[Synset('domestic_cat.n.01'), Synset('wildcat.n.03')]

Wordnet 사용해서 synset 간 의미론적 유사도 계산하기 (0-1 사이 실수), 
0이면 두 단어 서로 관계 없음,
1이면 완전 유의어

In [ ]:
# 고양이, 링스 : 스라소니 유사도 계산
x = wn.synset('cat.n.01') # 고양이
y = wn.synset('lynx.n.01') # 스라소니

# x,y 유사도 계산
x.path_similarity(y)

NameError: ignored

In [ ]:
x = wn.synset('cat.n.01') # 고양이
y = wn.synset('domestic_cat.n.01') # 집고양이
x.path_similarity(y)

0.5

In [ ]:
x = wn.synset('cat.n.01') # 고양이
y = wn.synset('cat.n.01') # 집고양이
x.path_similarity(y)

1.0

In [ ]:
x = wn.synset('cat.n.01') # 고양이
y = wn.synset('dog.n.01') # 집고양이
x.path_similarity(y)

0.2

In [ ]:
wn.synset('dog.n.01').hypernyms() # hypernyms 상위어

[Synset('canine.n.02'), Synset('domestic_animal.n.01')]

In [ ]:
wn.synset('dog.n.01').hyponyms() # hyponyms 하위어

[Synset('basenji.n.01'),
 Synset('corgi.n.01'),
 Synset('cur.n.01'),
 Synset('dalmatian.n.02'),
 Synset('great_pyrenees.n.01'),
 Synset('griffon.n.02'),
 Synset('hunting_dog.n.01'),
 Synset('lapdog.n.01'),
 Synset('leonberg.n.01'),
 Synset('mexican_hairless.n.01'),
 Synset('newfoundland.n.01'),
 Synset('pooch.n.01'),
 Synset('poodle.n.01'),
 Synset('pug.n.01'),
 Synset('puppy.n.01'),
 Synset('spitz.n.01'),
 Synset('toy_dog.n.01'),
 Synset('working_dog.n.01')]

자연어 처리 정규화 

In [ ]:
# 1. 토큰화 (텍스트 -> 단어 로 쪼갠다.)

from nltk.tokenize import WordPunctTokenizer
word_punct = WordPunctTokenizer()

text = "}Help! :))) :[ ..... :D{"

# Tokenizer눈 모든 구두점(punctuation; 문장부호)을 기준으로 분리
# -> 이모티콘을 이용한 감성 분석 등 문장구조 깊이 분석할 때 사용

word_punct.tokenize(text)

['}', 'Help', '!', ':)))', ':[', '.....', ':', 'D', '{']

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# 단어 토크나이저
nltk.word_tokenize(text)

['}', 'Help', '!', ':', ')', ')', ')', ':', '[', '...', '..', ':', 'D', '{']

In [ ]:
# 2. 단어의 대소문자를 통일한다 (전부 다 대문자 or 소문자)
# 3. 불용어 제거(stopwords 리스트 참조, The 같은 것들 제거)
# 4. 형태소 분석(stemming) 단어를 형태소로 변환

# 포터 형태소 분석기(보수적)
pstemmer = nltk.PorterStemmer()
pstemmer.stem("wonderful")

'wonder'

In [ ]:
# 랭커스터 형태소 분석기(적극적) - 더 많은 동음이의어 형태소 생산
lstemmer = nltk.LancasterStemmer()
lstemmer.stem("wonderful")

'wond'

In [ ]:
# 5. 원형 추출
lemmatizer = nltk.WordNetLemmatizer()
lemmatizer.lemmatize("wonderful")

'wonderful'

In [ ]:
# 그외 품사 태깅
nltk.download('averaged_perceptron_tagger')
nltk.pos_tag(["beautiful","go"])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('beautiful', 'NN'), ('go', 'VB')]

index.html 파일에서(불용어 제외) 가장 많이 등장한 단어 원형 찾아보기

In [ ]:
# www.networksciencelab.com 사이트에서 하이퍼 링크 정보 모두 가져오기

from bs4 import BeautifulSoup
from urllib.request import urlopen

with urlopen("http://www.networksciencelab.com/") as doc :
  soup = BeautifulSoup(doc)

# 하이퍼링크만 추출
links = [(link.string,link["href"]) for link in soup.find_all('a') if link.has_attr("href")]

# 책 제목만 추출
html_text = [i[0] for i in links] # i[0]은 책 제목 i[1]은 책 링크

# None 데이터 제거
html_text = " ".join(html_text[2:]) # 리스트 -> 하나의 텍스트로 변환
html_text

'DZPYDS DZCNAPY Networks of Music Groups as Success Predictors Network Science Workshop Resilience in Transaction-Oriented Networks Peer Ratings in Massive Online Social Networks Semantic Networks of Interests in Online NSSI Communities Towards an Ideal Store D.Zinoviev, "Analyzing Cultural Domains with Python," D. Zinoviev, D. Stefanescu, G. Fireman, and L. Swenson, "Semantic networks of interests in online non-suicidal self-injury communities," D.Zinoviev, "The Pain of Complexity," D.Zinoviev, Z.Zhu, and K.Li, "Building mini-categories in product networks," D.Zinoviev, H.Benbrahim, G.Meszoely, and D.Stefanescu, "Mitigation of delayed management costs in transaction-oriented systems," D.Zinoviev, H.Benbrahim, G.Meszoely, and D.Stefanescu, "Simulating resilience in transaction-oriented networks," D.Zinoviev, D.Stefanescu, L.Swenson, and G.Fireman, "Semantic networks of interests in online NSSI communities," D.Zinoviev and S.Llewelyn, "Co-Evolution of Friendship and Publishing in Online

In [ ]:
# 하나의 텍스트 -> index.html 파일로 저장
html_file = open("index.html","w")
html_file.write(html_text)
html_file.close() # 파일을 꼭 닫아주어야 한다.

index.html 파일에서(불용어 제외) 가장 많이 등장한 단어 원형 찾아보기

In [ ]:
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk import LancasterStemmer

# 형태소 분류기를 생성한다.
ls = nltk.LancasterStemmer()

# 파일을 읽고 soup을 만든다.
with open("index.html") as infile :
  soup = BeautifulSoup(infile)

In [ ]:
soup # tag들이 같이 출력된다

<html><body><p>DZPYDS DZCNAPY Networks of Music Groups as Success Predictors Network Science Workshop Resilience in Transaction-Oriented Networks Peer Ratings in Massive Online Social Networks Semantic Networks of Interests in Online NSSI Communities Towards an Ideal Store D.Zinoviev, "Analyzing Cultural Domains with Python," D. Zinoviev, D. Stefanescu, G. Fireman, and L. Swenson, "Semantic networks of interests in online non-suicidal self-injury communities," D.Zinoviev, "The Pain of Complexity," D.Zinoviev, Z.Zhu, and K.Li, "Building mini-categories in product networks," D.Zinoviev, H.Benbrahim, G.Meszoely, and D.Stefanescu, "Mitigation of delayed management costs in transaction-oriented systems," D.Zinoviev, H.Benbrahim, G.Meszoely, and D.Stefanescu, "Simulating resilience in transaction-oriented networks," D.Zinoviev, D.Stefanescu, L.Swenson, and G.Fireman, "Semantic networks of interests in online NSSI communities," D.Zinoviev and S.Llewelyn, "Co-Evolution of Friendship and Publis

In [ ]:
soup.text

'DZPYDS DZCNAPY Networks of Music Groups as Success Predictors Network Science Workshop Resilience in Transaction-Oriented Networks Peer Ratings in Massive Online Social Networks Semantic Networks of Interests in Online NSSI Communities Towards an Ideal Store D.Zinoviev, "Analyzing Cultural Domains with Python," D. Zinoviev, D. Stefanescu, G. Fireman, and L. Swenson, "Semantic networks of interests in online non-suicidal self-injury communities," D.Zinoviev, "The Pain of Complexity," D.Zinoviev, Z.Zhu, and K.Li, "Building mini-categories in product networks," D.Zinoviev, H.Benbrahim, G.Meszoely, and D.Stefanescu, "Mitigation of delayed management costs in transaction-oriented systems," D.Zinoviev, H.Benbrahim, G.Meszoely, and D.Stefanescu, "Simulating resilience in transaction-oriented networks," D.Zinoviev, D.Stefanescu, L.Swenson, and G.Fireman, "Semantic networks of interests in online NSSI communities," D.Zinoviev and S.Llewelyn, "Co-Evolution of Friendship and Publishing in Online

In [ ]:
# 1) 텍스트 추출 -> 토큰화
words = nltk.word_tokenize(soup.text)
words

In [ ]:
# 2) 단어를 소문자로 변환
words = [w.lower() for w in words]
words

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# 3) 불용어를 제거하고 단어의 형태소를 추출
# 3-1) 불용어 제거 stopwords.words("english")
# 3-2) 특수기호 제거 w.isalnum : alnum 문자열이 알파벳([a-zA-Z])고 숫자([0-9])로만 구성되어 있니?
# 3-3) 형태소 추출 ls.stem
words = [ls.stem(w) for w in words if w not in stopwords.words("english") and w.isalnum()]
words

In [ ]:
# 4) 가장 빈번하게 등장하는 단어 10개 추출
freqs = Counter(words)
print(freqs.most_common(5))

[('network', 16), ('soc', 8), ('onlin', 7), ('inform', 4), ('gam', 4)]
